ETL using BSE api and loading to mysql  


In [1]:
import pandas as pd
from bsedata.bse import BSE 
import time

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

excel_stocks = "https://github.com/suhascp7/bse_py/raw/refs/heads/master/Equity.xlsx"
engine_excel = 'openpyxl'
stockDf = pd.read_excel(excel_stocks,engine=engine_excel)
stockDf.head(2)

,Security Code,Issuer Name,Security Id,Security Name,Status,Group,Face Value,ISIN No,Industry,Instrument,Sector Name,Industry New Name,Igroup Name,ISubgroup Name
0,500002,ABB India Limited,ABB,ABB India Limited,Active,A,2.0,INE117A01022,Heavy Electrical Equipment,Equity,Industrials,Capital Goods,Electrical Equipment,Heavy Electrical Equipment
1,500003,Aegis Logistics Ltd.,AEGISLOG,AEGIS LOGISTICS LTD.,Active,A,1.0,INE208C01025,Trading - Gas,Equity,Energy,"Oil, Gas & Consumable Fuels",Gas,Trading - Gas


Getting nifty50, from the whole market 

In [2]:
# listOf_Nifty50_Stocks = [ 
#     "ADANIENT", "ADANIPORTS", "APOLLOHOSP", "ASIANPAINT", "AXISBANK",
#     "BAJAJ-AUTO", "BAJFINANCE", "BAJAJFINSV", "BPCL", "BHARTIARTL",
#     "BRITANNIA", "CIPLA", "COALINDIA", "DIVISLAB", "DRREDDY", "EICHERMOT",
#     "GRASIM", "HCLTECH", "HDFCBANK", "HDFCLIFE", "HEROMOTOCO", "HINDALCO",
#     "HINDUNILVR", "ICICIBANK", "ITC", "INDUSINDBK", "INFY", "JSWSTEEL",
#     "KOTAKBANK", "LTIM", "LT", "M&M", "MARUTI", "NTPC", "NESTLEIND",
#     "ONGC", "POWERGRID", "RELIANCE", "SBILIFE", "SBIN", "SUNPHARMA",
#     "TCS", "TATACONSUM", "TATAMOTORS", "TATASTEEL", "TECHM", "TITAN",
#     "UPL", "ULTRACEMCO", "WIPRO"
# ]

listOf_Nifty50_Stocks = [ 
    "ADANIENT", "ADANIPORTS", "ASIANPAINT", "AXISBANK",
    "BAJAJ-AUTO", "BAJFINANCE", "BAJAJFINSV", "BPCL", "BHARTIARTL",
     "CIPLA", "COALINDIA", "DIVISLAB", "DRREDDY", 
    "GRASIM", "HCLTECH", "HDFCBANK", "HDFCLIFE", "HINDALCO",
    "HINDUNILVR", "ICICIBANK", "ITC", "INDUSINDBK", "INFY", "JSWSTEEL",
    "KOTAKBANK", "LTIM", "LT", "M&M", "MARUTI", "NTPC", "NESTLEIND",
    "ONGC", "POWERGRID", "RELIANCE", "SBILIFE", "SBIN", "SUNPHARMA",
    "TCS", "TATACONSUM", "TATAMOTORS", "TATASTEEL", "TECHM"
   
]



stockDf['Security Code']= stockDf['Security Code'].astype(str)
nifty50 = stockDf[stockDf['Security Id'].isin(listOf_Nifty50_Stocks)].reset_index(drop=True)
nifty50.columns = nifty50.columns.str.replace(' ', '')


In [3]:
bseObject = BSE(update_codes=True) 

In [4]:
listof_stocksDict= []
seqCode = nifty50['SecurityCode']
for code in seqCode:
    try:
        dictStock = bseObject.getQuote(code)
        dictStock.pop('sell')
        dictStock.pop('buy')
        listof_stocksDict.append(dictStock)
        time.sleep(0.5)
    except IndexError:
        print(f'IndexError for {code}, data not available')


nifty50Today = pd.DataFrame(listof_stocksDict)

In [5]:
nifty50Today.head(2)

,companyName,currentValue,change,pChange,updatedOn,securityID,scripCode,group,faceValue,industry,previousClose,previousOpen,dayHigh,dayLow,52weekHigh,52weekLow,weightedAvgPrice,totalTradedValue,totalTradedQuantity,2WeekAvgQuantity,marketCapFull,marketCapFreeFloat
0,Bajaj Finance Limited,6716.90,44.35,0.66,04 Dec 24 | 11:22 AM,BAJFINANCE,500034,A / BSE SENSEX,2.00,Financial Services,6672.55,6680.05,6754.10,6675.85,7829.95,6190.00,6716.54,4.95 Cr.,0.07 Lakh,0.36 Lakh,"4,15,773.64 Cr.","1,87,098.14 Cr."
1,CIPLA LTD.,1512.10,-22.30,-1.45,04 Dec 24 | 11:23 AM,CIPLA,500087,A / BSE 100,2.00,Healthcare,1534.40,1533.20,1540.15,1506.50,1702.00,1192.85,1518.76,3.46 Cr.,0.23 Lakh,0.42 Lakh,"1,22,119.09 Cr.","84,262.17 Cr."


In [6]:
nifty50Today.rename(columns={'group': 'sharegroup'}, inplace=True)
nifty50Today.rename(columns={'52weekHigh': 'fiftytwoweekHigh'}, inplace=True)
nifty50Today.rename(columns={'52weekLow': 'fiftytwoweekLow'}, inplace=True)
nifty50Today.rename(columns={'2WeekAvgQuantity': 'twoWeekAvgQuantity'}, inplace=True)

# Converting 'updatedOn'  to datetime and extract date
nifty50Today['updatedOn'] = pd.to_datetime(nifty50Today['updatedOn'], format='%d %b %y | %I:%M %p', errors='coerce')

# Check if there are any invalid or missing date values
if pd.isna(nifty50Today['updatedOn']).any():
    print("There are invalid or missing date values in the 'updatedOn' column.")
else:
    # Extract date from 'updatedOn' column and convert the column to datetime
    nifty50Today['updatedOn'] = pd.to_datetime(nifty50Today['updatedOn'].dt.date)

if 'totalTradedValueCr' not in nifty50Today.columns:
   # Assuming nifty50Today is your DataFrame
    nifty50Today['totalTradedValueCr'] = pd.to_numeric(nifty50Today['totalTradedValue'].str.replace(',', '').str.replace(' Cr.', '', regex=True), errors='coerce')  # Convert to numeric and handle 'Cr.'
    nifty50Today['totalTradedQuantityLakh'] = pd.to_numeric(nifty50Today['totalTradedQuantity'].str.replace(',', '').str.replace(' Lakh', '', regex=True), errors='coerce')  # Convert to numeric and handle 'Lakh'
    nifty50Today['twoWeekAvgQuantityLakh'] = pd.to_numeric(nifty50Today['twoWeekAvgQuantity'].str.replace(',', '').str.replace(' Lakh', '', regex=True), errors='coerce')  # Convert to numeric and handle 'Lakh'
    nifty50Today['marketCapFullCr'] = pd.to_numeric(nifty50Today['marketCapFull'].str.replace(',', '').str.replace(' Cr.', '', regex=True), errors='coerce')  # Convert to numeric and handle 'Cr.'
    nifty50Today['marketCapFreeFloatCr'] = pd.to_numeric(nifty50Today['marketCapFreeFloat'].str.replace(',', '').str.replace(' Cr.', '', regex=True), errors='coerce')  # Convert to numeric and handle 'Cr.'

    # Drop original columns
    nifty50Today.drop(['totalTradedValue', 'totalTradedQuantity','twoWeekAvgQuantity', 'marketCapFull', 'marketCapFreeFloat'], axis=1, inplace=True)

nifty50Today.head(n=2)

,companyName,currentValue,change,pChange,updatedOn,securityID,scripCode,sharegroup,faceValue,industry,previousClose,previousOpen,dayHigh,dayLow,fiftytwoweekHigh,fiftytwoweekLow,weightedAvgPrice,totalTradedValueCr,totalTradedQuantityLakh,twoWeekAvgQuantityLakh,marketCapFullCr,marketCapFreeFloatCr
0,Bajaj Finance Limited,6716.90,44.35,0.66,2024-12-04,BAJFINANCE,500034,A / BSE SENSEX,2.00,Financial Services,6672.55,6680.05,6754.10,6675.85,7829.95,6190.00,6716.54,4.95,0.07,0.36,415773.64,187098.14
1,CIPLA LTD.,1512.10,-22.30,-1.45,2024-12-04,CIPLA,500087,A / BSE 100,2.00,Healthcare,1534.40,1533.20,1540.15,1506.50,1702.00,1192.85,1518.76,3.46,0.23,0.42,122119.09,84262.17


In [21]:
for row in nifty50Today.iterrows():
    print(row.values)

AttributeError: 'tuple' object has no attribute 'values'

In [ ]:
import mysql.connector

destination = {
    'host':'localhost',
    'database':'testdb',
    'user':'root',
    'password':'#2002#'
}
    

def get_connection():
    mydb = mysql.connector.connect(**destination)
    cursor = mydb.cursor()
    return mydb,cursor

try:
    mydb,cursor = get_connection()
    
    create_table_query= f"""
                        create table if not exists nifty50_daily (
                            companyName VARCHAR(255),
                            currentValue FLOAT,
                            `change` FLOAT,
                            pChange FLOAT,
                            updatedOn DATE,
                            securityID VARCHAR(255),
                            scripCode VARCHAR(255),
                            sharegroup VARCHAR(255),
                            faceValue FLOAT,
                            industry VARCHAR(255),
                            previousClose FLOAT,
                            previousOpen FLOAT,
                            dayHigh FLOAT,
                            dayLow FLOAT,
                            fiftytwoweekHigh FLOAT,
                            fiftytwoweekLow FLOAT,
                            weightedAvgPrice FLOAT,
                            totalTradedQuantityLakh FLOAT,
                            totalTradedValueCr FLOAT,
                            twoWeekAvgQuantityLakh FLOAT,
                            marketCapFullCr FLOAT,
                            marketCapFreeFloatCr FLOAT
                        );
    """

    cursor.execute(create_table_query)
    mydb.commit()


    cursor.execute("select max(updatedOn) from nifty50_daily")
    max_updateOn = cursor.fetchone()[0]
    print("max updated on in database",max_updateOn)

    df_max_updateOn = nifty50Today['updatedOn'].max()
    print("updateOn in dataframe is",df_max_updateOn)


    if(max_updateOn is None) or (df_max_updateOn > pd.Timestamp(max_updateOn)):
        for row in nifty50Today.iterrows():
            insert_query = f"""
            INSERT INTO nifty50_daily (
                companyName, currentValue, `change`, pChange, updatedOn, securityID, scripCode, sharegroup, 
                faceValue, industry, previousClose, previousOpen, dayHigh, dayLow, fiftytwoweekHigh, fiftytwoweekLow,
                weightedAvgPrice, totalTradedQuantityLakh, totalTradedValueCr, twoWeekAvgQuantityLakh, 
                marketCapFullCr, marketCapFreeFloatCr
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """
            cursor.execute(insert_query, tuple(row))                                                   #tuple(row.values)
           
        mydb.commit()
        print("New data inserted successfully.")
    else:
     print("up to date, no new data inserted") 

except mysql.connector.Error as e:
   print("sql error",e)

finally:
   cursor.close()
   mydb.close()






max updated on in database 2024-12-04
updateOn in dataframe is 2024-12-04 00:00:00
up to date, no new data inserted
